In [1]:
import torch
from torch import nn
from config import Config
from data.load import TableDataLoader
from load import load
from model.cmha import CompressedMultiHeadAttention
args = Config('./configs/ncgm.yaml')

In [3]:
tr_dl = TableDataLoader(args, "train")
table_train_dl = tr_dl.get_dataloader()

2023-04-10 21:26:29.252 | DEBUG    | data.scitsr:__init__:28 - Word2vec loaded from /home/data3/tanknee/CodeRepo/ncgm/data/word2vec/GoogleNews-vectors-negative300.bin
2023-04-10 21:26:29.382 | INFO     | data.load:get_dataloader:27 - Dataset SciTSR loaded


In [2]:
model, criterion, optimizer = load(args)

In [4]:
for batch in table_train_dl:
    # geometry, appearance, content, bounding_box, structure = batch
    # save batch pkl
    torch.save(batch, 'batch.pkl')
    # model(geometry, appearance, content, bounding_box)
    break

/home/fintech/anaconda3/envs/nee-start/lib/python3.9/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [5]:
batch = torch.load('batch.pkl')
geometry, appearance, content, bounding_box, row_adj_matrix, col_adj_matrix, cell_adj_matrix, structure = batch


In [6]:
bounding_box

tensor([[[  0, 188, 206, 324, 213],
         [  1,  80, 224, 122, 232],
         [  2, 262, 224, 447, 232],
         ...,
         [497,   0, 194,   0, 194],
         [498,   0, 194,   0, 194],
         [499,   0, 194,   0, 194]]])

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
bounding_box.shape

torch.Size([1, 500, 5])

In [7]:
cell_output, row_output, col_output, emb_pairs = model(geometry, appearance, content, bounding_box)

AssertionError: The boxes tensor shape is not correct as Tensor[K, 5]

In [ ]:
criterion(cell_output, row_output, col_output, emb_pairs, structure)

In [ ]:
cmha = CompressedMultiHeadAttention(args=args)
g = torch.randn(1, 500, 64)
a = torch.randn(1, 500 ** 2, 64)
c = torch.randn(1, 500 ** 2, 64)
o = cmha(g, a, c)